# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [17]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [3]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [39]:
# Carica il dataset Wine Quality White
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione
    
def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    #Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    if hyperparams['use_pca']:
        # ...
        if hyperparams['pca_standardize']:
            SC = StandardScaler() #istanzio uno scaler
            X_train = SC.fit_transform(X_train)
            X_val = SC.transform(X_val)
        P1 = PCA(hyperparams['pca_components'])
        X_train = P1.fit_transform(X_train)
        X_val = P1.transform(X_val)
        print("Ho utilizzato PCA con", hyperparams['pca_components'])
    if hyperparams['data_standardize']:
        # ...
        SC = StandardScaler()
        X_train = SC.fit_transform(X_train)
        X_val = SC.transform(X_val)
    # Aggiunge il termine costante ai dati
    # ...
    train_len = len(y_train)
    train_ones = np.ones(train_len)
    val_len = len(y_val)
    val_ones = np.ones(val_len)
    X_train = np.c_[X_train, train_ones]
    X_val = np.c_[X_val, val_ones]
    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        # ...
        LR = Ridge(hyperparams['reg_lambda']) 
        LR.fit(X_train, y_train)
    else:
        # ...
        LR = LinearRegression()
        LR.fit(X_train, y_train)
    # Calcolo predizioni
    # ...
    y_train_predict = LR.predict(X_train)
    y_val_predict = LR.predict(X_val)
    # Calcola il MAE
    # ...
    Mae_train = mean_absolute_error(y_train, y_train_predict)
    Mae_val = mean_absolute_error(y_val, y_val_predict)
    return Mae_train, Mae_val

# Dividi il dataset in training e test set
# ...
PTr = 0.6
PTot= 0.4
PVal = 0.5
PTest = 0.5
NumObs = len(y)
NumTrain = int(NumObs * 0.6)
NumTot = (NumObs - NumTrain)
X_train = X[:NumTrain]
y_train = y[:NumTrain]
XT = X[NumTrain:NumTrain+NumTot]
yT = y[NumTrain:NumTrain+NumTot]
# Dividi il training set in training set effettivo e validation set
# ...
NumVal = int(NumTot * PVal)
NumTest = NumTot - NumVal
X_val = XT[:NumVal]
y_val = yT[:NumVal]
X_test = XT[NumVal:NumTest+NumVal]
y_test = yT[NumVal:NumTest+NumVal]
print(NumObs, X.shape, X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)
# Trova la configurazione di iperparametri migliore
# ...
min_mae_val = 10000
curr_mae_val = 0
curr_mae_train = 0
best_id = -1
for i in range (0, len(configs)):
    curr_mae_train, curr_mae_val = pipeline(X_train, y_train, X_val, y_val, configs[i])
    print("Istanza:", i, curr_mae_train, curr_mae_val, "\n")
    if(curr_mae_val < min_mae_val):
        min_mae_val = curr_mae_val
        best_id = i
print("Il miglior mae:", min_mae_val, "appartiene alla", best_id, "configurazione")
# Riallena il modello sul training set completo
# ...
#per riallenare il modello uso gli iperparametri trovati, ridefinisco la funzione di pipeline eseguendo solo la prima parte
#del codice
X_train = X[:NumTrain+NumVal]
y_train = y[:NumTrain+NumVal]
choice = configs[best_id]
X_test[NumTrain+NumVal:NumTrain+NumVal+NumTest]
y_test[NumTrain+NumVal:NumTrain+NumVal+NumTest]
mae_train2, mae_test = pipeline(X_train, y_train, X_test, y_test, choice)
# Calcola il MAE sul test set
# ...
# Stampa  risultati
# ...
print("Il mae del test set così ottenuto vale:", mae_test)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


4898 (4898, 11) (2938, 11) (980, 11) (980, 11) (2938,) (980,) (980,)
Ho utilizzato PCA con 5
Istanza: 0 0.6310842223324808 0.6158409385560742 

Ho utilizzato PCA con 5
Istanza: 1 0.6408323673738419 0.6319822829262788 

Istanza: 2 0.6060127585436599 0.5933386781131156 

Ho utilizzato PCA con 10
Istanza: 3 0.6031960537003939 0.5896212781179913 

Ho utilizzato PCA con 10
Istanza: 4 0.6014512626660377 0.5910555941048616 

Ho utilizzato PCA con 10
Istanza: 5 0.6032038234132127 0.5896022509937149 

Ho utilizzato PCA con 10
Istanza: 6 0.6032159381683632 0.5896847537640251 

Ho utilizzato PCA con 3
Istanza: 7 0.6474365905132156 0.6543357915564225 

Ho utilizzato PCA con 3
Istanza: 8 0.6474366765672386 0.6543350409467925 

Ho utilizzato PCA con 3
Istanza: 9 0.6914631811098183 0.6691229956882868 

Istanza: 10 0.5975729956803988 0.5894207956411487 

Ho utilizzato PCA con 5
Istanza: 11 0.6311032092112063 0.6157418081589421 

Ho utilizzato PCA con 3
Istanza: 12 0.647437286366081 0.6543283566407697 